# सत्र २ – मिनिमल RAG पाइपलाइन

Foundry Local + sentence-transformers embeddings वापरून एक हलकी Retrieval-Augmented Generation पाइपलाइन तयार करा.


### स्पष्टीकरण: अवलंबित्व स्थापना
या पाइपलाइनसाठी आवश्यक असलेल्या किमान पॅकेजेसची स्थापना:
- स्थानिक मॉडेल व्यवस्थापनासाठी `foundry-local-sdk` (जर शुद्ध BASE_URL पथाचा वापर करत नसाल तर).
- सुसंगत SDK संरचनांसाठी `openai` (काही उपयुक्तता).
- एम्बेडिंगसाठी `sentence-transformers`.
- व्हेक्टर गणितासाठी `numpy`.
पुन्हा चालवणे सुरक्षित; जर वातावरण आधीच पूर्ण झाले असेल तर वगळा.


# परिस्थिती
हे नोटबुक पूर्णपणे स्थानिक पातळीवर चालणारी एक साधी Retrieval-Augmented Generation (RAG) पाइपलाइन तयार करते:
- Foundry Local मॉडेलशी कनेक्ट होते (SDK किंवा BASE_URL द्वारे स्वयंचलित शोध).
- एक लहान इन-मेमरी दस्तऐवज संग्रह तयार करते आणि त्याला Sentence Transformers च्या मदतीने एम्बेड करते.
- पारदर्शकतेसाठी साधा व्हेक्टर समानता पुनर्प्राप्ती (कोणताही बाह्य निर्देशांक नाही) अंमलात आणते.
- अनेक HTTP फॉलबॅक मार्गांद्वारे (`/v1/chat/completions`, `/v1/completions`, `/v1/responses`) आधारभूत जनरेशन विनंत्या करते.
- एक `answer()` सहाय्यक प्रदान करते जो प्रारंभिक प्रयत्न अयशस्वी झाल्यास पर्यायी मॉडेल फॉर्म पुन्हा प्रयत्न करते.

मोठ्या संग्रहांमध्ये, स्थिर व्हेक्टर स्टोअर्स किंवा मूल्यांकन मेट्रिक्समध्ये स्केल करण्यापूर्वी (RAG मूल्यांकन नोटबुक पहा) याचा निदान टेम्पलेट म्हणून वापर करा.


In [5]:
# Install dependencies
!pip install -q foundry-local-sdk openai sentence-transformers numpy

### स्पष्टीकरण: मुख्य आयात
एम्बेडिंग + स्थानिक अनुमानासाठी आवश्यक मुख्य लायब्ररी लोड करते:
- SentenceTransformer साठी घन व्हेक्टर एम्बेडिंग.
- FoundryLocalManager (पर्यायी) स्थानिक सेवा व्यवस्थापित करण्यासाठी.
- OpenAI क्लायंट परिचित ऑब्जेक्ट आकारांसाठी (जरी नंतर आम्ही थेट HTTP वापरतो).


In [6]:
import os, numpy as np
from sentence_transformers import SentenceTransformer
from foundry_local import FoundryLocalManager
from openai import OpenAI

### स्पष्टीकरण: खेळणी दस्तऐवज संग्रह
डोमेन विधानांची एक छोटी इन-मेमरी यादी परिभाषित करते. पुनरावृत्ती जलद आणि नियंत्रित ठेवते जेणेकरून लक्ष डेटा व्यवस्थापनाऐवजी पाइपलाइन यांत्रिकी (पुनर्प्राप्ती + ग्राउंडिंग) वर केंद्रित राहते.


In [7]:
DOCS = [
    'Foundry Local provides an OpenAI-compatible local inference endpoint.',
    'Retrieval Augmented Generation improves answer grounding by injecting relevant context.',
    'Edge AI reduces latency and preserves privacy via local execution.',
    'Small Language Models can offer competitive quality with lower resource usage.',
    'Vector similarity search retrieves semantically relevant documents.'
]

### स्पष्टीकरण: कनेक्शन, मॉडेल निवड आणि एम्बेडिंग प्रारंभ
मजबूत कनेक्शन लॉजिक:
1. वैकल्पिकरित्या स्पष्ट `BASE_URL` (शुद्ध HTTP मार्ग) वापरते, अन्यथा FoundryLocalManager वर परत जाते.
2. `/v1/models` ची चाचणी करते आणि सर्वात चांगल्या जुळणाऱ्या ठोस मॉडेल आयडीची निवड करते (अचूक उपनाम > कॅनॉनिकल फॅमिली > प्रथम उपलब्ध).
3. `FOUNDRY_CONNECT_RETRIES` आणि विलंबासह पुनर्प्रयत्न लूप अंमलात आणते.
4. टॉय कॉर्पससाठी SentenceTransformer एम्बेडिंग (सामान्यीकृत व्हेक्टर) प्रारंभ करते.
5. पुनरुत्पादकतेसाठी OpenAI SDK आवृत्ती कॅप्चर करते.
सेवा अनुपस्थित असल्यास, क्रॅश होण्याऐवजी ती सुरू करण्यासाठी मार्गदर्शन प्रिंट करते.


In [12]:
import os, time, json, requests, re
# Native Foundry Local SDK preferred; fall back to explicit BASE_URL if provided
os.environ.setdefault('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini')
alias = os.getenv('FOUNDRY_LOCAL_ALIAS', os.getenv('TARGET_MODEL', 'phi-4-mini'))
base_url_env = os.getenv('BASE_URL', '').strip()
manager = None
client = None
endpoint = None

def _canonicalize(model_id: str) -> str:
    """Remove CUDA suffix and version tags from model name."""
    b = model_id.split(':')[0]
    return re.sub(r'-cuda.*', '', b)

try:
    if base_url_env:
        # Allow user override; normalize by removing trailing / and optional /v1
        root = base_url_env.rstrip('/')
        if root.endswith('/v1'):
            root = root[:-3]
        endpoint = root
        print(f'[INFO] Using explicit BASE_URL override: {endpoint}')
    else:
        from foundry_local import FoundryLocalManager
        manager = FoundryLocalManager(alias)
        # Manager endpoint already includes /v1 - remove it for our base
        raw_endpoint = manager.endpoint.rstrip('/')
        if raw_endpoint.endswith('/v1'):
            endpoint = raw_endpoint[:-3]
        else:
            endpoint = raw_endpoint
        print(f'[OK] Foundry Local manager endpoint: {manager.endpoint} | base={endpoint} | alias={alias}')
    
    # Probe models list (endpoint does NOT include /v1 here)
    models_resp = requests.get(endpoint + '/v1/models', timeout=5)
    models_resp.raise_for_status()
    payload = models_resp.json() if models_resp.headers.get('content-type','').startswith('application/json') else {}
    data = payload.get('data', []) if isinstance(payload, dict) else []
    ids = [m.get('id') for m in data if isinstance(m, dict)]
    
    # Select best matching model
    chosen = None
    if alias in ids:
        chosen = alias
    else:
        for mid in ids:
            if _canonicalize(mid) == _canonicalize(alias):
                chosen = mid
                break
    if not chosen and ids:
        chosen = ids[0]
    model_name = chosen or alias
    
    # Initialize OpenAI client
    from openai import OpenAI as _OpenAI
    client = _OpenAI(
        base_url=endpoint + '/v1',  # OpenAI client needs full base URL with /v1
        api_key=(getattr(manager, 'api_key', None) or os.getenv('API_KEY') or 'not-needed')
    )
    print(f'[OK] Model resolved: {model_name} (total_models={len(ids)})')
except Exception as e:
    print('[ERROR] Failed to initialize Foundry Local client:', e)
    client = None
    model_name = alias

# Expose BASE for downstream compatibility (without /v1)
BASE = endpoint

# Embeddings setup
embed_model_name = os.getenv('EMBED_MODEL', 'sentence-transformers/all-MiniLM-L6-v2')
try:
    from sentence_transformers import SentenceTransformer
    embedder = SentenceTransformer(embed_model_name)
    doc_emb = embedder.encode(DOCS, convert_to_numpy=True, normalize_embeddings=True)
    print(f'[OK] Embedded {len(DOCS)} docs using {embed_model_name} shape={doc_emb.shape}')
except Exception as e:
    print('[ERROR] Embedding init failed:', e)
    embedder = None
    doc_emb = None

try:
    import openai as _openai
    openai_version = getattr(_openai, '__version__', 'unknown')
    print('OpenAI SDK version:', openai_version)
except Exception:
    openai_version = 'unknown'

if client is None:
    print('\nNEXT: Start/verify service then re-run this cell:')
    print('  foundry service start')
    print('  foundry model run phi-4-mini')
    print('  (optional) set BASE_URL=http://127.0.0.1:57127')

[OK] Foundry Local manager endpoint: http://127.0.0.1:59778/v1 | base=http://127.0.0.1:59778 | alias=phi-4-mini
[OK] Model resolved: deepseek-r1-distill-qwen-7b-cuda-gpu:0 (total_models=11)
[OK] Embedded 5 docs using sentence-transformers/all-MiniLM-L6-v2 shape=(5, 384)
OpenAI SDK version: 1.109.1


### स्पष्टीकरण: Retrieve फंक्शन (व्हेक्टर साम्य)
`retrieve(query, k=3)` क्वेरी एन्कोड करते, कोसाइन साम्य (सामान्यीकृत व्हेक्टरवर डॉट प्रॉडक्ट) गणना करते आणि टॉप-k डॉक इंडेक्स परत करते. हे पारदर्शकतेसाठी साधे आणि इन-मेमरी ठेवले जाते.


In [9]:
def retrieve(query, k=3):
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)[0]
    sims = doc_emb @ q
    return sims.argsort()[::-1][:k]

### स्पष्टीकरण: SDK-आधारित जनरेशन आणि उत्तर सहाय्यक
Foundry Local SDK + OpenAI-सुसंगत क्लायंट पद्धतींचा वापर करून पुन्हा डिझाइन केले:
- प्राथमिक मार्ग: `client.chat.completions.create` (संरचित संदेश).
- पर्याय: `client.completions.create` (जुना प्रॉम्प्ट) नंतर `client.responses.create` (सुलभ प्रतिसाद API).
- पर्यायी मॉडेल आयडी (RAW विरुद्ध स्ट्रिप्ड ALT) सामान्य करून सुसंगतता वाढवते.
- `answer()` टॉप-k पुनर्प्राप्त डॉक्युमेंट्समधून एक आधारभूत प्रॉम्प्ट तयार करते आणि प्रयत्नांच्या क्रमबद्ध नोंदी ठेवते.
हे लॉजिक वाचनीय ठेवते आणि तरीही OpenAI-सुसंगत एन्डपॉइंट्सच्या उत्क्रांतीमध्ये सहजतेने कार्य करण्याची क्षमता प्रदान करते.


In [14]:
# SDK-based generation (Foundry Local manager + OpenAI client methods)
import re, time, json

def _strip_model_name(name: str) -> str:
    """Strip CUDA suffix and version tags from model name."""
    base = name.split(':')[0]
    base = re.sub(r'-cuda.*', '', base)
    return base

# Use the actual resolved model name from connection cell
RAW_MODEL = model_name
ALT_MODEL = _strip_model_name(RAW_MODEL)

def _try_via_client(messages, prompt, model_id: str, max_tokens=220, temperature=0.2):
    """Try generating response using OpenAI client with multiple fallback routes."""
    attempts = []
    
    # 1. Try chat.completions endpoint (preferred for chat models)
    try:
        resp = client.chat.completions.create(
            model=model_id, 
            messages=messages, 
            max_tokens=max_tokens, 
            temperature=temperature
        )
        content = resp.choices[0].message.content
        attempts.append(('chat.completions', 200, (content or '')[:160]))
        if content and content.strip():
            return content, attempts
    except Exception as e:
        attempts.append(('chat.completions', None, str(e)[:160]))
    
    # 2. Try legacy completions endpoint
    try:
        comp = client.completions.create(
            model=model_id, 
            prompt=prompt, 
            max_tokens=max_tokens, 
            temperature=temperature
        )
        txt = comp.choices[0].text if comp.choices else ''
        attempts.append(('completions', 200, (txt or '')[:160]))
        if txt and txt.strip():
            return txt, attempts
    except Exception as e:
        attempts.append(('completions', None, str(e)[:160]))
    
    return None, attempts

def retrieve(query, k=3):
    """Retrieve top-k most similar documents using cosine similarity."""
    if embedder is None or doc_emb is None:
        raise RuntimeError("Embeddings not initialized.")
    q_emb = embedder.encode([query], normalize_embeddings=True)[0]
    scores = doc_emb @ q_emb
    idxs = np.argsort(scores)[::-1][:k]
    return idxs

def answer(query, k=3, max_tokens=220, temperature=0.2, try_alternate=True):
    """
    Answer a query using RAG pipeline:
    1. Retrieve relevant documents using vector similarity
    2. Generate grounded response using Foundry Local model via OpenAI SDK
    
    Args:
        query: User question
        k: Number of documents to retrieve
        max_tokens: Maximum tokens for generation
        temperature: Sampling temperature
        try_alternate: Whether to try alternate model name on failure
    
    Returns:
        Dictionary with query, answer, docs, context, route, and tried attempts
    """
    if client is None:
        raise RuntimeError('Model client not initialized. Re-run connection cell after starting Foundry Local.')
    if embedder is None or doc_emb is None:
        raise RuntimeError('Embeddings not initialized.')
    
    # Retrieve relevant documents
    idxs = retrieve(query, k=k)
    context = '\n'.join(f'Doc {i}: {DOCS[i]}' for i in idxs)
    
    # Construct grounded generation prompt
    system_content = 'Use ONLY provided context. If insufficient, say "I\'m not sure."'
    user_content = f'Context:\n{context}\n\nQuestion: {query}'
    messages = [
        {'role': 'system', 'content': system_content},
        {'role': 'user', 'content': user_content}
    ]
    prompt = f'System: {system_content}\n{user_content}\nAnswer:'
    
    # Try generation with primary model
    tried = []
    ans, attempts = _try_via_client(messages, prompt, RAW_MODEL, max_tokens=max_tokens, temperature=temperature)
    tried.append({'model': RAW_MODEL, 'attempts': attempts})
    
    if ans and ans.strip():
        return {
            'query': query, 
            'answer': ans.strip(), 
            'docs': idxs.tolist(), 
            'context': context, 
            'route': 'chat-first', 
            'tried': tried
        }
    
    # Try alternate model name if available
    if try_alternate and ALT_MODEL != RAW_MODEL:
        ans2, attempts2 = _try_via_client(messages, prompt, ALT_MODEL, max_tokens=max_tokens, temperature=temperature)
        tried.append({'model': ALT_MODEL, 'attempts': attempts2})
        if ans2 and ans2.strip():
            return {
                'query': query, 
                'answer': ans2.strip(), 
                'docs': idxs.tolist(), 
                'context': context, 
                'route': 'chat-alt', 
                'tried': tried
            }
    
    # All routes failed
    return {
        'query': query, 
        'answer': 'I\'m not sure. (All SDK routes failed)', 
        'docs': idxs.tolist(), 
        'context': context, 
        'route': 'failed', 
        'tried': tried
    }

print('[INFO] SDK generation mode active.')
print(f'       RAW_MODEL = {RAW_MODEL}')
print(f'       ALT_MODEL = {ALT_MODEL}')

[INFO] SDK generation mode active.
       RAW_MODEL = deepseek-r1-distill-qwen-7b-cuda-gpu:0
       ALT_MODEL = deepseek-r1-distill-qwen-7b


In [15]:
# Self-test cell: validates connectivity, embeddings, and answer() basic functionality (SDK mode)
import math, pprint

def rag_self_test(sample_query: str = 'Why use RAG with local inference?', expect_docs: int = 3):
    report = {'base': BASE, 'raw_model': RAW_MODEL, 'alt_model': ALT_MODEL}
    if not BASE:
        report['error'] = 'BASE not resolved'
        return report
    if embedder is None or doc_emb is None:
        report['error'] = 'Embeddings not initialized'
        return report
    if getattr(doc_emb, 'shape', (0,))[0] != len(DOCS):
        report['warning_embeddings'] = f"doc_emb count {getattr(doc_emb,'shape',('?'))} mismatch DOCS {len(DOCS)}"
    try:
        idxs = retrieve(sample_query, k=expect_docs)
        report['retrieved_indices'] = idxs.tolist() if hasattr(idxs, 'tolist') else list(idxs)
    except Exception as e:
        report['error_retrieve'] = str(e)
        return report
    try:
        ans = answer(sample_query, k=expect_docs, max_tokens=80, temperature=0.2)
        report['route'] = ans.get('route')
        report['answer_preview'] = ans.get('answer','')[:160]
        if ans.get('route') == 'failed':
            report['warning_generation'] = 'All SDK routes failed for sample query'
    except Exception as e:
        report['error_generation'] = str(e)
    return report

pprint.pprint(rag_self_test())

{'alt_model': 'deepseek-r1-distill-qwen-7b',
 'answer_preview': 'Okay, so I need to figure out why someone would use '
                   'Retrieval Augmented Generation (RAG) with local inference. '
                   'Let me start by understanding each part of the qu',
 'base': 'http://127.0.0.1:59778',
 'raw_model': 'deepseek-r1-distill-qwen-7b-cuda-gpu:0',
 'retrieved_indices': [0, 3, 1],
 'route': 'chat-first'}


### स्पष्टीकरण: बॅच क्वेरी स्मोक टेस्ट
`answer()` च्या माध्यमातून अनेक प्रतिनिधिक वापरकर्ता प्रश्नांची अंमलबजावणी करते, यासाठी:
- पुनर्प्राप्ती निर्देशांक योग्य समर्थन दस्तऐवजांशी संबंधित आहेत.
- फॉलबॅक रूटिंग कार्य करते (रूट मूल्य 'failed' नाही).
- उत्तरांमध्ये ग्राउंडिंग सूचना पाळली जाते (कोणतेही काल्पनिक उत्तर नाही).
अधूनमधून तपासणीसाठी शेवटचा परिणाम ऑब्जेक्ट कॅप्चर करते.


In [16]:
# Quick test queries

queries = [

    "Why use RAG with local inference?",

    "What does vector similarity search do?",

    "Explain privacy benefits."

]



last_result = None

for q in queries:

    try:

        r = answer(q)

        last_result = r

        print(f"Q: {q}\nA: {r['answer']}\nDocs: {r['docs']}\n---")

    except Exception as e:

        print(f"Failed answering '{q}': {e}")



last_result

Q: Why use RAG with local inference?
A: Okay, so I need to figure out why someone would use Retrieval Augmented Generation (RAG) with local inference. Let me start by understanding each part of the question.

First, RAG. From the context given, Doc 1 says that RAG improves answer grounding by injecting relevant context. So RAG is a method that uses retrieval techniques to find the most relevant parts of a document or corpus to augment the generation process. This probably helps in making the generated answers more accurate because they're backed by real data.

Then, local inference. Doc 0 mentions that Foundry Local provides an OpenAI-compatible local inference endpoint. So local inference means running the model on the user's device rather than sending the request to a remote server. This is good for privacy and reducing latency, but it might have limitations in terms of model size or capabilities compared to cloud-based options.

Now, combining RAG with local inference. The context s

{'query': 'Explain privacy benefits.',
 'answer': 'Okay, so I need to explain the privacy benefits mentioned in the provided context. Let me look at the context again. The context includes three documents:\n\nDoc 2 says Edge AI reduces latency and preserves privacy via local execution.\nDoc 3 mentions Small Language Models can offer competitive quality with lower resource usage.\nDoc 1 states Retrieval Augmented Generation improves answer grounding by injecting relevant context.\n\nThe question is about explaining the privacy benefits. So, I should focus on the parts of the context that talk about privacy. \n\nLooking at Doc 2, it mentions Edge AI reduces latency and preserves privacy via local execution. That seems directly related to privacy. I think "local execution" means that the AI processes data on the device itself rather than sending it to a server. This could mean that data doesn\'t have to be transmitted, which might help protect user privacy because it avoids centralizing d

### स्पष्टीकरण: एकल उत्तर सोयीस्कर कॉल
सोप्या कॉपी/पेस्ट पुनर्वापरासाठी किंवा पुढील संदर्भासाठी अंतिम जलद एकल-प्रश्न कॉल. पूर्वीच्या वॉर्म-अप क्वेरीज नंतर `answer()` चा idempotent वापर कसा करायचा हे दाखवते.


In [17]:
result = answer('Why use RAG with local inference?')
result

{'query': 'Why use RAG with local inference?',
 'answer': "Okay, so I need to figure out why someone would use Retrieval Augmented Generation (RAG) with local inference. Let me start by understanding each part of the question.\n\nFirst, RAG. From the context given, Doc 1 says that RAG improves answer grounding by injecting relevant context. So RAG is a method that uses retrieval techniques to find the most relevant parts of a document or corpus to augment the generation process. This probably helps in making the generated answers more accurate because they're backed by real data.\n\nThen, local inference. Doc 0 mentions that Foundry Local provides an OpenAI-compatible local inference endpoint. So local inference means running the model on the user's device rather than sending the request to a remote server. This is good for privacy and reducing latency, but it might have limitations in terms of model size or capabilities compared to cloud-based options.\n\nNow, combining RAG with local


---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून भाषांतरित केला गेला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित भाषांतरांमध्ये चुका किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील मूळ दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी, व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर केल्यामुळे उद्भवलेल्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
